<a href="https://colab.research.google.com/github/Mohammad-Mjadleh/Statistical-Analysis-Project/blob/main/CreditScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# import

In [1]:
import sys
import ssl
import numpy as np  # a module for working with numerical array 
import pandas as pd  # a module for working with data-frames
import statsmodels.api as sm  # a module for statistical modelling (e.g. regression analysis)
from matplotlib import pyplot as plt
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from scipy import stats 
import io
from sklearn.model_selection import train_test_split
import warnings
import statsmodels.api as sm  # a module for statistical modelling (e.g. regression analysis)

warnings.filterwarnings('ignore')

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


# pre

In [134]:
train = pd.read_csv(io.BytesIO(uploaded['train.csv']),  encoding = 'iso-8859-8' , header= 0)
test = pd.read_csv(io.BytesIO(uploaded['test.csv']),  encoding = 'iso-8859-8' , header= 0)


In [135]:
test.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [136]:
train = train[train.columns[2:]] # Removing metadata columns
train = train.drop(['Name','SSN'], axis=1)
test = test[test.columns[2:]] # Removing metadata columns
test = test.drop(['Name','SSN'], axis=1)

In [137]:
def to_months(age_str):
    age_list = age_str.split()
    years = int(age_list[0])
    months = int(age_list[3])
    total_months = years * 12 + months
    return total_months
train['Credit_History_Age']= train['Credit_History_Age'].fillna("0 Years and 0 Months")
train['Credit_History_Age'] = train['Credit_History_Age'].apply(to_months)
test['Credit_History_Age']= test['Credit_History_Age'].fillna("0 Years and 0 Months")
test['Credit_History_Age'] = test['Credit_History_Age'].apply(to_months)


In [138]:
cols = ['Month' , 'Occupation', 'Payment_Behaviour', 'Payment_of_Min_Amount','Type_of_Loan','Credit_Mix']

In [139]:
from sklearn.preprocessing import LabelEncoder

# Label encode features
label_encoder = LabelEncoder()

train['Credit_Score'] = label_encoder.fit_transform(train['Credit_Score'])

for i in cols :
  train[i] = label_encoder.fit_transform(train[i])
  test[i] = label_encoder.fit_transform(test[i])

# Print sample of dataset
train.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,3,23,12,19114.12,1824.843333,3,4,3,4,128,...,3,809.98,26.822620,265,1,49.574949,80.41529543900253,3,312.49408867943663,0
1,2,23,12,19114.12,NaN,3,4,3,4,128,...,1,809.98,31.944960,0,1,49.574949,118.28022162236736,4,284.62916249607184,0
2,6,-500,12,19114.12,NaN,3,4,3,4,128,...,1,809.98,28.609352,267,1,49.574949,81.699521264648,5,331.2098628537912,0
3,0,23,12,19114.12,NaN,3,4,3,4,128,...,1,809.98,31.377862,268,1,49.574949,199.4580743910713,6,223.45130972736786,0
4,7,23,12,19114.12,1824.843333,3,4,3,4,128,...,1,809.98,24.797347,269,1,49.574949,41.420153086217326,2,341.48923103222177,0


In [140]:
train.describe()

,Month,Occupation,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Type_of_Loan,Delay_from_due_date,Num_Credit_Inquiries,Credit_Mix,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Credit_Score
count,100000.000000,100000.000000,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,3.500000,7.523380,4194.170850,17.091280,22.47443,72.466040,3513.769280,21.068780,27.754251,1.578800,32.285173,201.221460,1.403190,1403.118217,3.469040,1.353460
std,2.291299,4.640296,3183.686167,117.404834,129.05741,466.422621,1975.132865,14.860104,193.177339,1.013647,5.116875,114.320699,0.693378,8306.041270,2.035471,0.764913
min,0.000000,0.000000,303.645417,-1.000000,0.00000,1.000000,0.000000,-5.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.750000,4.000000,1625.568229,3.000000,4.00000,8.000000,1777.750000,10.000000,3.000000,1.000000,28.052567,114.000000,1.000000,30.306660,2.000000,1.000000
50%,3.500000,7.000000,3093.745000,6.000000,5.00000,13.000000,3463.000000,18.000000,6.000000,2.000000,32.305784,208.000000,2.000000,69.249473,3.000000,2.000000
75%,5.250000,12.000000,5957.448333,7.000000,7.00000,20.000000,5385.250000,28.000000,9.000000,2.000000,36.496663,292.000000,2.000000,161.224249,6.000000,2.000000
max,7.000000,15.000000,15204.633333,1798.000000,1499.00000,5797.000000,6260.000000,67.000000,2597.000000,3.000000,50.000000,404.000000,2.000000,82331.000000,6.000000,2.000000


# addititonal pre + ols model

section2

In [141]:
import re

def extract_number(obj):
    if isinstance(obj, str):
        match = re.search(r'\d+', obj)
        if match:
            return int(match.group())
    elif isinstance(obj, (int, float)):
        return obj


In [142]:
cols = train.select_dtypes(exclude='number').columns
for i in cols :
  train[i] = train[i].apply(extract_number)
  train[i] = pd.to_numeric(train[i]) 
  test[i] = test[i].apply(extract_number)
  test[i] = pd.to_numeric(test[i])





In [143]:
TEMP = train.dropna()
Xinfo = TEMP.loc[:, TEMP.columns!='Credit_Score']
Y = TEMP.loc[:, TEMP.columns =='Credit_Score']


In [144]:
model = sm.OLS(Y, Xinfo)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           Credit_Score   R-squared:                      -3.158
Model:                            OLS   Adj. R-squared:                 -3.158
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 09 Apr 2023   Prob (F-statistic):                nan
Time:                        09:09:51   Log-Likelihood:            -1.3333e+05
No. Observations:               71615   AIC:                         2.667e+05
Df Residuals:                   71614   BIC:                         2.667e+05
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Month                   

In [145]:
cov_mat = TEMP.cov()

cov_mat

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
Month,5.241122e+00,-1.202177e+00,6.389725e-03,2.061458e+04,5.383258e+00,-8.491353e-01,6.288457e-01,-1.235412e+00,-3.056130e-01,5.611206e+00,...,-2.785513e-03,1.175277e+00,4.074640e-03,2.181007e-01,-2.439939e-03,-3.336163e+01,1.569303e+01,2.908273e-02,-6.952985e+21,7.619635e-03
Age,-1.202177e+00,4.744360e+05,-8.894507e+00,-1.089890e+06,1.029266e+04,1.674906e+02,-2.774897e+02,-1.009167e+03,-3.862835e+01,-4.578736e+03,...,-3.813793e+00,-4.466879e+02,2.427918e+01,7.825057e+00,-1.927845e+00,-4.746831e+03,1.016075e+04,-5.469846e+00,-2.733244e+24,1.964578e-02
Occupation,6.389725e-03,-8.894507e+00,2.149473e+01,3.688562e+03,1.514991e+01,-2.440246e+00,-1.633715e+00,5.242312e-01,-1.162193e+00,-2.453573e+01,...,2.174180e-02,3.778937e+01,6.391214e-02,-4.509394e-01,-1.406764e-02,4.618331e+01,9.149207e+01,3.262374e-02,-5.887179e+22,2.387653e-02
Annual_Income,2.061458e+04,-1.089890e+06,3.688562e+03,2.057645e+12,1.263504e+08,-5.811620e+05,4.881417e+05,-3.423935e+06,6.236960e+02,1.153310e+07,...,-8.240411e+03,2.678929e+05,9.636390e+04,-6.060786e+05,-6.351618e+02,-3.338715e+07,-1.007074e+07,-2.579455e+04,-4.235560e+27,4.776824e+03
Monthly_Inhand_Salary,5.383258e+00,1.029266e+04,1.514991e+01,1.263504e+08,1.001830e+07,-3.170007e+03,-2.321607e+03,-1.105372e+04,-2.484877e+03,4.395101e+05,...,1.871714e+02,-9.865958e+05,2.952402e+03,8.047673e+04,-4.256873e+02,2.152545e+05,3.549290e+05,-1.736397e+03,-7.238452e+24,-1.795340e+02
Num_Bank_Accounts,-8.491353e-01,1.674906e+02,-2.440246e+00,-5.811620e+05,-3.170007e+03,1.342565e+04,-1.854622e+01,-1.557840e+02,-7.977118e+00,-1.284407e+03,...,-2.316967e-02,2.330776e+03,-4.109950e+00,-1.241835e+02,9.261839e-01,-8.889388e+02,5.926828e+02,1.831832e-01,-4.078571e+23,3.155013e-01
Num_Credit_Card,6.288457e-01,-2.774897e+02,-1.633715e+00,4.881417e+05,-2.321607e+03,-1.854622e+01,1.639231e+04,-3.693718e+02,2.237785e+01,3.182659e+02,...,6.044396e-02,1.270335e+03,-2.171788e+00,-3.997559e+00,2.847657e-01,4.442105e+02,4.803339e+02,1.510620e+00,-5.461316e+23,-2.487566e-01
Interest_Rate,-1.235412e+00,-1.009167e+03,5.242312e-01,-3.423935e+06,-1.105372e+04,-1.557840e+02,-3.693718e+02,2.237632e+05,-9.639921e+01,-7.600245e+03,...,-1.381063e+00,4.301780e+03,1.734299e-01,-2.960415e+02,1.204775e+00,2.313769e+04,-1.775580e+03,1.978463e+00,-2.029070e+24,-1.414476e+00
Num_of_Loan,-3.056130e-01,-3.862835e+01,-1.162193e+00,6.236960e+02,-2.484877e+03,-7.977118e+00,2.237785e+01,-9.639921e+01,3.835746e+03,-1.734053e+03,...,-7.454632e-01,1.593042e+03,-2.209662e+00,-1.347821e+02,3.130938e-01,-2.119249e+03,3.153207e+02,2.785031e-02,-2.303705e+23,4.409608e-02
Type_of_Loan,5.611206e+00,-4.578736e+03,-2.453573e+01,1.153310e+07,4.395101e+05,-1.284407e+03,3.182659e+02,-7.600245e+03,-1.734053e+03,3.893606e+06,...,7.178313e+01,-2.574189e+05,3.256383e+02,2.282944e+04,-9.803815e+01,6.756233e+03,-5.610653e+03,-7.202390e+01,-1.172296e+25,-2.819909e+01


In [170]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingRegressor



x_tr ,x_te, y_tr , y_te = train_test_split(Xinfo, Y , test_size = 0.2)

model = HistGradientBoostingClassifier()
model.fit(x_tr,y_tr)
prediction = model.predict(x_te)


score = accuracy_score(y_te, prediction)


model2 = DecisionTreeClassifier()
model2.fit(x_tr,y_tr)
prediction2 = model2.predict(x_te)


score2 = accuracy_score(y_te, prediction2)

model3 = RandomForestClassifier()
model3.fit(x_tr,y_tr)
prediction3 = model3.predict(x_te)


score3 = accuracy_score(y_te, prediction3)



In [174]:
score

0.7331564616351323

In [175]:
score2

0.6727640857362285

In [176]:
score3

0.7790965579836626



```
# This is formatted as code
```

# trial
